# NeuraChat: Smart Mental Health Companion

## Description:
An empathetic chatbot designed to provide emotional support, mindfulness exercises, and crisis management.

## How It Works:
- Users can share their feelings (e.g., "I'm feeling stressed"), and the chatbot provides tailored mindfulness exercises or motivational quotes.
- Tracks user mood over time using short-term memory.
- For critical situations, escalates the conversation to a human counselor.

## LangGraph Topics Covered:
- **State Schemas**: Define workflows for different mental health scenarios (e.g., stress, anxiety, motivation).
- **Conditional Edges**: Route conversations based on sentiment (positive, neutral, negative).
- **Short-Term Memory**: Track user mood and personalize interactions.
- **Streaming**: Provide real-time mindfulness exercises.

## LangSmith Topics Covered:
- Debug workflows for crisis management using breakpoints.
- Test chatbot responses for empathy and relevance using tracing tools.



#  Imports
### Import necessary libraries for the chatbot workflow.

In [2]:
from enum import Enum
import ollama
import logging  
from typing import List, Dict
from pydantic import BaseModel
from langgraph.graph import StateGraph



#  Logging Configuration
### Configure logging to track events and errors.

In [3]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")



# Sentiment Enum
### Define an enumeration for sentiment analysis categories.

In [4]:
class Sentiment(str, Enum):
    NEGATIVE = "negative"
    POSITIVE = "positive"
    NEUTRAL = "neutral"


# ChatState Model
### This class represents the state of the chatbot, including messages, sentiment, summary, feedback, and user input.


In [5]:
class ChatState(BaseModel):
    messages: List[Dict[str, str]] = []  
    sentiment: Sentiment = Sentiment.NEUTRAL  
    summary: str = ""  
    feedback: str = ""  
    user_input: str = ""  


# Sentiment Analysis Function
### This function analyzes the sentiment of a given message.

In [6]:
def analyze_sentiment(message: str) -> Sentiment:
    try:
        if any(word in message.lower() for word in ["sad", "depressed", "anxious", "upset"]):
            return Sentiment.NEGATIVE
        elif any(word in message.lower() for word in ["happy", "excited", "joyful"]):
            return Sentiment.POSITIVE
        return Sentiment.NEUTRAL
    except Exception as e:
        logging.error(f"Error analyzing sentiment: {e}")
        return Sentiment.NEUTRAL

# Chatbot Response Function
### This function generates a chatbot response using Ollama and updates the state.


In [7]:
def chatbot_response(state: ChatState) -> ChatState:
    user_input = state.user_input.strip()  # Extract user input from state

    if not user_input:
        logging.warning("Empty user input received.")
        return state  # Return state unchanged
    
    try:
        response = ollama.chat(model="llama3", messages=[{"role": "user", "content": user_input}])

        new_state = state.model_copy()
        new_state.messages.append({"user": user_input, "bot": response["message"]["content"]})
        new_state.sentiment = analyze_sentiment(user_input)

        return new_state

    except Exception as e:
        logging.error(f"Error in chatbot response: {e}")
        return state  # Return previous state if an error occurs


# Conversation Summarization Function
### This function summarizes the chat history.


In [8]:
def summarize_conversation(state: ChatState) -> ChatState:
    try:
        messages_text = "\n".join([msg["user"] + " " + msg["bot"] for msg in state.messages])
        summary = ollama.chat(model="llama3", messages=[{"role": "user", "content": "Summarize this conversation: " + messages_text}])

        new_state = state.model_copy()
        new_state.summary = summary["message"]["content"]

        return new_state
    except Exception as e:
        logging.error(f"Error summarizing conversation: {e}")
        return state  # Return previous state if an error occurs


#  Sentiment Routing Function
### This function determines the chatbot's response path based on sentiment.


In [9]:
def route_by_sentiment(state: ChatState):
    if state.sentiment == Sentiment.NEGATIVE:
        return "negative_response"
    elif state.sentiment == Sentiment.POSITIVE:
        return "positive_response"
    return "neutral_response"



#  Workflow Definition
### Define the chatbot workflow using LangGraph.

In [10]:
workflow = StateGraph(ChatState)

workflow.add_node("chatbot", chatbot_response)
workflow.add_node("summarize", summarize_conversation)
workflow.add_node("negative_response", chatbot_response)
workflow.add_node("positive_response", chatbot_response)
workflow.add_node("neutral_response", chatbot_response)

# ✅ **Register `route_by_sentiment` as a decision node**
workflow.add_conditional_edges("chatbot", route_by_sentiment)

# ✅ **Connect all sentiment responses to summarization**
workflow.add_edge("negative_response", "summarize")
workflow.add_edge("positive_response", "summarize")
workflow.add_edge("neutral_response", "summarize")

workflow.set_entry_point("chatbot")
graph = workflow.compile()

# Running the Chatbot
### The chatbot runs in a loop, taking user input and generating responses until interrupted.


In [ ]:
if __name__ == "__main__":
    state = ChatState()
    while True:
        try:
            user_input = input("You: ")
            state.user_input = user_input  # ✅ Store user input in state
        
            # 🔹 Run decision-based workflow
            result = graph.invoke(state)
            if isinstance(result, ChatState):
                state = result
            else:
                state = ChatState(**result)

            logging.info(f"Bot: {state.messages[-1]['bot']}")
            logging.info(f"[Mood: {state.sentiment}]\n")
        except KeyboardInterrupt:
            logging.info("Chatbot terminated by user.")
            logging.info("\nGoodbye!")
            break
        except Exception as e:
            logging.error(f"Unexpected error in main loop: {e}")


2025-03-16 02:12:10,034 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-16 02:13:18,407 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-16 02:16:12,037 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-16 02:16:12,043 - INFO - Bot: I'm so sorry to hear that you're feeling sad. It's totally okay to feel that way, and I'm here to listen and offer some support.

Can you tell me a little bit more about what's going on and why you're feeling down? Sometimes talking about it can help, or we can brainstorm together to find ways to lift your mood.

Remember, you're not alone, and I'm here for you. Sending you lots of positive vibes and a big virtual hug!
2025-03-16 02:16:12,045 - INFO - [Mood: Sentiment.NEGATIVE]

2025-03-16 02:17:36,703 - WARNING - Empty user input received.
2025-03-16 02:17:36,708 - WARNING - Empty user input received.
